In [76]:
import pandas as pd

In [77]:
reference_data=pd.read_csv("D:/EDrivebackup/brocamp/DataScienceProjects/bw2_prj/HealthMonitoring/data/merged_patient_data.csv")

In [78]:
current_data=pd.read_csv("D:/EDrivebackup/brocamp/DataScienceProjects/bw2_prj/HealthMonitoring/artifacts/merged_patient_kafka_data.csv")

In [79]:
!pip install evidently==0.6.4



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, RegressionPreset

In [100]:

from IPython.display import display

import json


import os

class EvidentlyMonitor:
    def __init__(self, report_dir="reports"):
        
        self.report_dir = report_dir
        os.makedirs(self.report_dir, exist_ok=True)

    def generate_report(self, baseline_df: pd.DataFrame, current_df: pd.DataFrame, title: str):
       
        
        report = Report([DataDriftPreset(drift_share=0.7)])
        report.run(reference_data=baseline_df, current_data=current_df)

        report_file = os.path.join(self.report_dir, f"{title.lower().replace(' ', '_')}.html")
        report.save_html(report_file)
        snapshot_dict = report.as_dict()  
        print(snapshot_dict)
        json_file = os.path.join(self.report_dir, f"{title.lower().replace(' ', '_')}_report.json")
        with open(json_file, 'w') as f:
            json.dump(snapshot_dict, f, indent=4)
        report.show()
        
        
        
#     def generate_report(self, baseline_df: pd.DataFrame, current_df: pd.DataFrame, title: str):
#         report = Report(metrics=[DataDriftPreset()])
#         my_eval=report.run(reference_data=baseline_df, current_data=current_df)

#         report_file = os.path.join(self.report_dir, f"{title.lower().replace(' ', '_')}.html")
#         snapshot_dict = my_eval.json()
#         print(snapshot_dict)
#         report.show()
        

       




In [101]:
monitor = EvidentlyMonitor()

In [83]:
current_data.columns

Index(['Patient_ID', 'Date', 'Time', 'Blood Glucose Level (mg/dL)',
       'Blood Oxygen (SpO₂)', 'Electrocardiogram (ECG/EKG)',
       'Hydration Levels', 'patient_id_x', 'timestamp_x', 'Blood Pressure',
       'Heart Rate (HR)', 'Respiratory Rate (RR)', 'Body Temperature',
       'patient_id_y', 'timestamp_y', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [84]:
current_data[:1]

,Patient_ID,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,patient_id_x,timestamp_x,Blood Pressure,...,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,P12332,2025-03-01,0.57.33,145.197576,96.81888,0.883574,64.832462,P12332,1746262840226,129/83,...,247.433986,64,295839,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545


In [85]:
reference_data['Date']

0       02-03-2025
1       02-03-2025
2       02-03-2025
3       02-03-2025
4       02-03-2025
           ...    
2587    28-02-2025
2588    28-02-2025
2589    28-02-2025
2590    28-02-2025
2591    28-02-2025
Name: Date, Length: 2592, dtype: object

In [86]:
reference_data.columns



Index(['Patient_ID', 'Date', 'Time', 'Blood Glucose Level (mg/dL)',
       'Blood Oxygen (SpO₂)', 'Electrocardiogram (ECG/EKG)',
       'Hydration Levels', 'Blood Pressure', 'Heart Rate (HR)',
       'Respiratory Rate (RR)', 'Body Temperature', 'Hemoglobin', 'Glucose',
       'Cholesterol', 'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count',
       'Creatinine', 'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [87]:
current_data.columns

Index(['Patient_ID', 'Date', 'Time', 'Blood Glucose Level (mg/dL)',
       'Blood Oxygen (SpO₂)', 'Electrocardiogram (ECG/EKG)',
       'Hydration Levels', 'patient_id_x', 'timestamp_x', 'Blood Pressure',
       'Heart Rate (HR)', 'Respiratory Rate (RR)', 'Body Temperature',
       'patient_id_y', 'timestamp_y', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [88]:
current_data.dtypes

Patient_ID                      object
Date                            object
Time                            object
Blood Glucose Level (mg/dL)    float64
Blood Oxygen (SpO₂)            float64
Electrocardiogram (ECG/EKG)    float64
Hydration Levels               float64
patient_id_x                    object
timestamp_x                      int64
Blood Pressure                  object
Heart Rate (HR)                  int64
Respiratory Rate (RR)            int64
Body Temperature               float64
patient_id_y                    object
timestamp_y                      int64
Hemoglobin                     float64
Glucose                        float64
Cholesterol                    float64
Heart Rate                       int64
Platelet Count                   int64
WBC Count                      float64
RBC Count                      float64
Creatinine                     float64
Urea                           float64
Sodium                         float64
Potassium                

In [89]:
current_data[['Systolic', 'Diastolic']] = current_data['Blood Pressure'].str.split('/', expand=True).astype(float)
current_data.drop('Blood Pressure', axis=1, inplace=True)
reference_data[['Systolic', 'Diastolic']] = reference_data['Blood Pressure'].str.split('/', expand=True).astype(float)
reference_data.drop('Blood Pressure', axis=1, inplace=True)



In [90]:
reference_data=reference_data.drop(['Date','Time','Patient_ID'],axis=1)
current_data=current_data.drop(['Date','Time','Patient_ID','timestamp_y','timestamp_x','patient_id_y','patient_id_x'],axis=1)

In [91]:
current_data.shape

(250, 21)

In [92]:
reference_data.shape

(2592, 21)

In [93]:
numerical_features=reference_data.columns

In [94]:
columnmapping=ColumnMapping()
columnmapping.numerical_features=numerical_features

In [102]:
monitor.generate_report(reference_data,current_data,title="Input Feature Drift")

{'metrics': [{'metric': 'DatasetDriftMetric', 'result': {'drift_share': 0.7, 'number_of_columns': 21, 'number_of_drifted_columns': 13, 'share_of_drifted_columns': 0.6190476190476191, 'dataset_drift': False}}, {'metric': 'DataDriftTable', 'result': {'number_of_columns': 21, 'number_of_drifted_columns': 13, 'share_of_drifted_columns': 0.6190476190476191, 'dataset_drift': True, 'drift_by_columns': {'Blood Glucose Level (mg/dL)': {'column_name': 'Blood Glucose Level (mg/dL)', 'column_type': 'num', 'stattest_name': 'Wasserstein distance (normed)', 'stattest_threshold': 0.1, 'drift_score': 0.08539324570102852, 'drift_detected': False, 'current': {'small_distribution': {'x': [60.5, 74.43715407, 88.37430814, 102.31146221, 116.24861628, 130.18577034999998, 144.12292442, 158.06007849, 171.99723256, 185.93438663, 199.8715407], 'y': [0.0034440316695157247, 0.004879044865147282, 0.006314058060778829, 0.0080360738955367, 0.008610079173789321, 0.0068880633390314425, 0.009758089730294564, 0.0077490712